In [1]:
import numpy as np
import pandas as pd

In [13]:
edges = pd.read_csv('data/edges.txt', sep='\t', header=None)

In [122]:
nodes = pd.read_csv('data/nodes.txt', sep='\t', header=None)

In [38]:
nodes

,0,1,2,3
0,1,100monkeystyping.com,0,Blogarama
1,2,12thharmonic.com/wordpress,0,BlogCatalog
2,3,40ozblog.blogspot.com,0,"Blogarama,BlogCatalog"
3,4,4lina.tblog.com,0,Blogarama
4,5,750volts.blogspot.com,0,Blogarama
...,...,...,...,...
1485,1486,youngconservative.blogspot.com,1,Blogarama
1486,1487,zebrax.blogs.com,1,BlogCatalog
1487,1488,zeke01.blogspot.com,1,"Blogarama,BlogCatalog"
1488,1489,zeke01.typepad.com,1,Blogarama


In [19]:
edges

,0,1
0,267,1394
1,267,483
2,267,1051
3,904,1479
4,904,919
...,...,...
19085,1133,1390
19086,1133,1429
19087,1133,1423
19088,1133,1408


In [30]:
edges_list = []

for i, j in edges.values:
    
    if (i, j) not in edges_list or (j, i) not in edges_list:
        edges_list.append((i, j))
        
unique_edges = pd.DataFrame(edges_list)

In [177]:
unique_edges

,0,1
0,267,1394
1,267,483
2,267,1051
3,904,1479
4,904,919
...,...,...
19075,1133,1390
19076,1133,1429
19077,1133,1423
19078,1133,1408


In [45]:
len(nodes[0].unique())

1490

In [244]:
node_degree = {}
for i in nodes[0].unique():
    node_degree[i] = 0

In [245]:
for i, j in edges_list:
    node_degree[i] += 1
    node_degree[j] += 1

In [246]:
m = pd.DataFrame(node_degree, index=node_degree.keys())

In [248]:
D = np.zeros(m.shape, int)
np.fill_diagonal( D, np.diagonal(m ))
D.shape

(1490, 1490)

In [249]:
A = np.zeros(m.shape, int)
for i, j in edges_list:
    A[i-1][j-1] = 1
    A[j-1][i-1] = 1

In [250]:
idx = np.where(D.sum(axis=0) != 0)[0]


In [251]:
A = A[idx, :]
A = A[:, idx]

In [252]:
A.shape

(1224, 1224)

In [254]:
node_degree = {k:v for k, v in node_degree.items() if v != 0}
m = pd.DataFrame(node_degree, index=node_degree.keys())
D = np.zeros(m.shape, int)
np.fill_diagonal( D, np.diagonal(m ))
D.shape

(1224, 1224)

In [255]:
A

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [256]:
L = D - A
L

array([[27, -1,  0, ...,  0,  0,  0],
       [-1, 48,  0, ...,  0,  0,  0],
       [ 0,  0,  4, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  1, -1,  0],
       [ 0,  0,  0, ..., -1, 22,  0],
       [ 0,  0,  0, ...,  0,  0,  1]])

In [272]:
k = 4

v, x= np.linalg.eig(L)

indx_sorted = np.argsort(v)
x = x[:, idx_sorted[-k:]].real

x = x/np.repeat(np.sqrt(np.sum(x*x, axis=1).reshape(-1, 1)), k, axis=1)

kmeans = KMeans(n_clusters=k).fit(x.real)
labels = kmeans.labels_

In [280]:
nodes_test = nodes.copy()
nodes_test = nodes_test[nodes_test[0].isin(node_degree.keys())]
nodes_test['label'] = labels
nodes_test['label'].value_counts()

2    411
0    371
1    270
3    172
Name: label, dtype: int64

In [281]:
nodes_test

,0,1,2,3,label
0,1,100monkeystyping.com,0,Blogarama,0
1,2,12thharmonic.com/wordpress,0,BlogCatalog,0
4,5,750volts.blogspot.com,0,Blogarama,2
5,6,95theses.blogspot.com,0,Blogarama,2
6,7,abbadabbaduo.blogspot.com,0,"Blogarama,LeftyDirectory",2
...,...,...,...,...,...
1485,1486,youngconservative.blogspot.com,1,Blogarama,3
1486,1487,zebrax.blogs.com,1,BlogCatalog,2
1487,1488,zeke01.blogspot.com,1,"Blogarama,BlogCatalog",2
1488,1489,zeke01.typepad.com,1,Blogarama,3


In [287]:
classification = nodes_test.groupby(['label'])[2].agg(lambda x: x.value_counts().index[0])

In [319]:
df = nodes_test.merge(classification, left_on='label', right_on='label')

In [320]:
df = df[[0, '2_x', 'label', '2_y']].rename(columns={0:'id','2_x':'truth', '2_y':'pred'})
df

,id,truth,label,pred
0,1,0,0,0
1,2,0,0,0
2,8,0,0,0
3,9,0,0,0
4,10,0,0,0
...,...,...,...,...
1219,1474,1,3,1
1220,1475,1,3,1
1221,1485,1,3,1
1222,1486,1,3,1


In [323]:
df = df.groupby(['truth', 'pred'])['label'].value_counts().reset_index(name='count').sort_values('label')

In [324]:
df

,truth,pred,label,count
0,0,0,0,351
5,1,0,0,20
2,0,1,1,19
6,1,1,1,251
1,0,0,2,215
4,1,0,2,196
3,0,1,3,3
7,1,1,3,169


In [327]:
df = df.merge(df.groupby('label')['count'].sum(), on='label')

In [328]:
df

,truth,pred,label,count_x,count_y
0,0,0,0,351,371
1,1,0,0,20,371
2,0,1,1,19,270
3,1,1,1,251,270
4,0,0,2,215,411
5,1,0,2,196,411
6,0,1,3,3,172
7,1,1,3,169,172


In [331]:
df['rate'] = df['count_x']/df['count_y']
df

,truth,pred,label,count_x,count_y,rate
0,0,0,0,351,371,0.946092
1,1,0,0,20,371,0.053908
2,0,1,1,19,270,0.070370
3,1,1,1,251,270,0.929630
4,0,0,2,215,411,0.523114
5,1,0,2,196,411,0.476886
6,0,1,3,3,172,0.017442
7,1,1,3,169,172,0.982558
